### Экологический мониторинг качества воздуха с использованием предобученных NLP-моделей

Данный ноутбук демонстрирует прототип IT-решения для мониторинга состояния окружающей среды на основе данных экологических сенсоров.
В работе используются синтетически сгенерированные данные, имитирующие показания городских станций (PM2.5, CO₂, температура, влажность, скорость ветра), и предобученные модели машинного обучения из библиотеки Hugging Face Transformers.

Пайплайн включает:

Генерацию синтетического набора данных для трёх районов города.

Базовую пороговую классификацию PM2.5 (clean / moderate / high / dangerous).

Определение тренда качества воздуха за последние 1.5 часа.

Формирование текстового описания состояния воздуха на английском языке.

Zero-shot классификацию экологической ситуации с помощью модели `facebook/bart-large-mnli`.

Агрегацию данных и генерацию суточного отчёта.

Сжатие отчёта (summarization) моделью `facebook/bart-large-cnn`.

Ноутбук иллюстрирует, как готовые предобученные модели NLP могут использоваться для анализа данных реального мира и формирования интерпретируемых отчётов без обучения собственных моделей.

In [ ]:
!pip install -U transformers accelerate

^C


  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/12.0 MB 10.9 MB/s eta 0:00:01
   ------------- -------------------------- 3.9/12.0 MB 9.5 MB/s eta 0:00:01
   ----------------------- ---------------- 7.1/12.0 MB 11.5 MB/s eta 0:00:01
   ------------------------------- -------- 9.4/12.0 MB 11.5 MB/s eta 0:00:01
   ---------------------------------------  11.8/12.0 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 11.3 MB/s  0:00:01
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB 9.1 MB/s  0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------  2.6/2.7 MB 12.8 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 11.4 MB/s  0:00:00
   -------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from transformers import pipeline

***Генерация синтетических данных сенсоров***

В этой ячейке создаётся искусственный набор данных, имитирующий работу городских экологических станций.
Для трёх районов города моделируются показания сенсоров (PM2.5, CO₂, температура, влажность, скорость ветра) с интервалом в 30 минут в течение одной недели.

При генерации используются:

базовые уровни показателей для каждого района, чтобы районы отличались друг от друга;

случайные колебания, имитирующие естественные изменения параметров;

редкие аномальные всплески PM2.5, которые создают реалистичные ситуации загрязнения воздуха.

Результат - таблица `df` с временным рядом значений сенсоров, на которой далее будут демонстрироваться методы анализа и классификации экологической ситуации.

In [ ]:
# параметры датасета
areas = ["North", "South", "Center"]
start_time = datetime(2025, 1, 1, 0, 0)
end_time   = datetime(2025, 1, 7, 23, 30)
freq = "30min"

time_index = pd.date_range(start=start_time, end=end_time, freq=freq)

rows = []
rng = np.random.default_rng(42)

for area in areas:
    # базовые "нормальные" уровни для района
    base_pm25 = rng.uniform(10, 35)
    base_co2  = rng.uniform(400, 800)
    base_temp = rng.uniform(-5, 5)
    base_hum  = rng.uniform(60, 85)

    for t in time_index:
        # небольшие случайные колебания
        pm25 = base_pm25 + rng.normal(0, 5)
        co2  = base_co2  + rng.normal(0, 50)
        temp = base_temp + 10 * np.sin(2 * np.pi * (t.hour / 24)) + rng.normal(0, 1)
        hum  = base_hum  + rng.normal(0, 3)
        wind = abs(rng.normal(2, 1))  # м/с

        # "аномальные всплески" PM2.5
        if rng.uniform() < 0.02:
            pm25 += rng.uniform(40, 120)

        rows.append({
            "timestamp": t,
            "area": area,
            "pm25": max(pm25, 1),
            "co2": max(co2, 300),
            "temperature": temp,
            "humidity": np.clip(hum, 20, 100),
            "wind_speed": wind
        })

df = pd.DataFrame(rows)
df.head()

,timestamp,area,pm25,co2,temperature,humidity,wind_speed
0,2025-01-01 00:00:00,North,19.593725,510.442401,3.713820,76.485473,1.983199
1,2025-01-01 00:30:00,North,33.745891,614.440973,3.652010,80.815924,2.467509
2,2025-01-01 01:00:00,North,31.192655,527.607246,7.052620,77.284423,1.815138
3,2025-01-01 01:30:00,North,35.461608,567.824902,5.745842,76.377800,2.532309
4,2025-01-01 02:00:00,North,31.412564,597.092426,10.727627,76.214956,1.487757


***Базовая классификация уровня загрязнения по PM2.5***

Здесь мы вводим простую пороговую логику для оценки качества воздуха по концентрации PM2.5.
Такая "ручная" классификация служит baseline-моделью, с которой позже будет сравниваться работа предобученной zero-shot модели.

Используем типичные пороги качества воздуха:

менее 15 - clean

15–35 - moderate

35–75 - high

выше 75 - dangerous

Полученный столбец `pm25_level_rule` показывает техническую оценку уровня загрязнения и помогает анализировать, насколько zero-shot классификация согласуется с реальными нормами.

In [ ]:
def pm25_level(pm25):
    """Пороговая классификация уровня загрязнения по PM2.5.

    Возвращает одну из категорий: "clean", "moderate", "high", "dangerous"
    на основе фиксированных порогов концентрации.
    """
    
    if pm25 < 15:
        return "clean"
    elif pm25 < 35:
        return "moderate"
    elif pm25 < 75:
        return "high"
    else:
        return "dangerous"

df["pm25_level_rule"] = df["pm25"].apply(pm25_level)
df["pm25_level_rule"].value_counts()

,count
pm25_level_rule,
moderate,900
clean,46
high,44
dangerous,18


***Определение тренда изменения PM2.5***

Для анализа экологической ситуации важно учитывать не только текущее значение PM2.5, но и его динамику во времени.
В этой ячейке мы вычисляем, как изменялась концентрация PM2.5 в каждом районе за последний короткий промежуток времени (1.5 часа). Классифицируем изменение как: rising fast, rising, stable, falling, falling fast.

Полученный признак `pm25_trend` помогает zero-shot модели лучше понимать ситуацию и позволяет описывать загрязнение в динамике, а не только по текущему измерению.

In [ ]:
# сортируем для вычисления тренда
df = df.sort_values(["area", "timestamp"]).reset_index(drop=True)

# считаем разность PM2.5 по времени внутри района
df["pm25_prev"] = df.groupby("area")["pm25"].shift(3)  # 3 шага = 1.5 часа
df["pm25_change"] = df["pm25"] - df["pm25_prev"]

def pm25_trend(change):
    """Классифицирует изменение PM2.5 за 1.5 часа.

    Параметр change — разность между текущим значением и значением 3 шага назад.
    Возвращает одну из категорий: "rising fast", "rising", "stable",
    "falling", "falling fast" или "unknown" для первых строк.
    """
    
    if pd.isna(change):
        return "unknown"
    if change > 15:
        return "rising fast"
    elif change > 5:
        return "rising"
    elif change < -15:
        return "falling fast"
    elif change < -5:
        return "falling"
    else:
        return "stable"

df["pm25_trend"] = df["pm25_change"].apply(pm25_trend)

***Формирование текстового описания состояния воздуха для NLP-модели***

На этом шаге мы переводим числовые показания сенсоров в понятное текстовое описание на английском языке (английский используется потому, что модель `facebook/bart-large-mnli` изначально обучена на английском), которое затем подаётся в zero-shot модель.

Для каждой строки данных:

учитываем район, значения PM2.5, CO₂, температуры, влажности и скорости ветра;

добавляем информацию о тренде PM2.5 (растёт, падает, стабилен);

используем пороговую оценку `pm25_level_rule`, чтобы словесно описать уровень загрязнения (clean / moderate / high / dangerous);

явно определяем риск для здоровья и необходимость предупреждений (нет риска, нужно наблюдать, вредно для чувствительных групп, опасно для всех).

В результате получаем колонку `description_en` - текстовое описание состояния воздуха, которое имитирует краткий комментарий и служит входом для предобученной zero-shot модели.

In [ ]:
def build_description(row):
    """Формирует текстовое описание состояния воздуха на английском языке.

    Использует:
    - пороговую категорию загрязнения pm25_level_rule;
    - значения PM2.5, CO2, температуры, влажности и скорости ветра;
    - тренд PM2.5 (pm25_trend);
    и возвращает осмысленный текст, который подаётся в zero-shot модель.
    """
    
    level = row["pm25_level_rule"]

    # Фразы для описания уровня загрязнения
    if level == "clean":
        level_phrase = (
            "This PM2.5 level is clearly below common safety guidelines, "
            "so the air can be considered clean."
        )
        risk_phrase = (
            "According to safety standards, the air poses no health risk. "
        )
    elif level == "moderate":
        level_phrase = (
            "This PM2.5 level is close to but still below the typical safety limit, "
            "so the air quality can be considered moderate."
        )
        risk_phrase = (
            "The air is generally safe, but the situation should be monitored. "
        )
    elif level == "high":
        level_phrase = (
            "This PM2.5 level is above the typical safety limit, "
            "so the air quality can be considered high pollution."
        )
        risk_phrase = (
            "This level of air pollution may be harmful for sensitive groups. "
        )
    else:  # dangerous
        level_phrase = (
            "This PM2.5 level is far above the typical safety limit and clearly dangerous "
            "for human health."
        )
        risk_phrase = (
            "This level of air pollution is dangerous for human health. "
            "A public warning is recommended. "
        )

    # Основное описание
    desc = (
        f"District: {row['area']}. "
        f"PM2.5 is {row['pm25']:.1f} micrograms per cubic meter. "
        f"CO2 level is {row['co2']:.0f} ppm. "
        f"Temperature is {row['temperature']:.1f} degrees Celsius. "
        f"Relative humidity is {row['humidity']:.0f} percent. "
        f"Wind speed is {row['wind_speed']:.1f} meters per second. "
    )

    # Тренд PM2.5
    if row["pm25_trend"] == "rising fast":
        desc += "PM2.5 is rising quickly in the last hour. "
    elif row["pm25_trend"] == "rising":
        desc += "PM2.5 is gradually increasing. "
    elif row["pm25_trend"] == "falling fast":
        desc += "PM2.5 is dropping quickly. "
    elif row["pm25_trend"] == "falling":
        desc += "PM2.5 is slowly decreasing. "
    else:
        desc += "PM2.5 is relatively stable. "

    # фразы про уровень и риск
    desc += level_phrase + " " + risk_phrase

    return desc


df["description_en"] = df.apply(build_description, axis=1)
df[["timestamp", "area", "pm25", "pm25_level_rule", "pm25_trend", "description_en"]].head(3)

,timestamp,area,pm25,pm25_level_rule,pm25_trend,description_en
0,2025-01-01 00:00:00,Center,23.070723,moderate,unknown,District: Center. PM2.5 is 23.1 micrograms per...
1,2025-01-01 00:30:00,Center,22.897061,moderate,unknown,District: Center. PM2.5 is 22.9 micrograms per...
2,2025-01-01 01:00:00,Center,21.332908,moderate,unknown,District: Center. PM2.5 is 21.3 micrograms per...


***Zero-shot классификация экологической ситуации по текстовому описанию***

На этом шаге мы подключаем предобученную модель `facebook/bart-large-mnli` и используем её в режиме `zero-shot classification`.
Модель получает текстовое описание состояния воздуха (`description_en`) и выбирает один из заранее заданных классов экологической ситуации: *clean air, moderate air, pollution, high air pollution, dangerous air pollution.*

Zero-shot классификация позволяет интерпретировать данные сенсоров так, как это делал бы эксперт, без дополнительного обучения модели.

Результат сохраняется в столбец `situation_zero_shot`, а уверенность модели - в `situation_score`.

In [ ]:
classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device_map="auto"
)

candidate_labels = [
    "clean air",
    "moderate air pollution",
    "high air pollution",
    "dangerous air pollution"
]

# применяем zero-shot классификацию ко всем текстовым описаниям
results = classifier(
    list(df["description_en"]),
    candidate_labels,
    multi_label=False
)

df["situation_zero_shot"] = [r["labels"][0] for r in results]
df["situation_score"] = [r["scores"][0] for r in results]

df[[
    "timestamp",
    "area",
    "pm25",
    "pm25_level_rule",
    "situation_zero_shot",
    "situation_score"
]].head(10)

Device set to use cuda:0


,timestamp,area,pm25,pm25_level_rule,situation_zero_shot,situation_score
0,2025-01-01 00:00:00,Center,23.070723,moderate,moderate air pollution,0.899401
1,2025-01-01 00:30:00,Center,22.897061,moderate,moderate air pollution,0.905446
2,2025-01-01 01:00:00,Center,21.332908,moderate,moderate air pollution,0.904546
3,2025-01-01 01:30:00,Center,19.257803,moderate,moderate air pollution,0.905654
4,2025-01-01 02:00:00,Center,19.380748,moderate,moderate air pollution,0.903137
5,2025-01-01 02:30:00,Center,21.596663,moderate,moderate air pollution,0.904017
6,2025-01-01 03:00:00,Center,96.497511,dangerous,dangerous air pollution,0.702035
7,2025-01-01 03:30:00,Center,21.287285,moderate,moderate air pollution,0.900564
8,2025-01-01 04:00:00,Center,20.110532,moderate,moderate air pollution,0.900707
9,2025-01-01 04:30:00,Center,27.219698,moderate,moderate air pollution,0.908574


***Формирование и краткое обобщение суточного отчёта по данным сенсоров***

В этой ячейке автоматически создаётся текстовый отчёт о качестве воздуха за один день в одном районе, а затем выполняется его сжатие с помощью предобученной summarization-модели.

Что делаем:

Добавляется колонка `date`, содержащая календарную дату каждого измерения.

Данные группируются по району и дню, и для каждой группы вычисляются ключевые показатели PM2.5:

– среднее значение,

– минимальное,

– максимальное.

На этом шаге используется весь датасет, так как агрегирование выполняется по всем районам и всем дням.

Из полученной таблицы выбирается одна суточная запись (первый район и первая дата), и на её основе формируется развёрнутый текстовый отчёт `raw_report`.

Предобученная модель `facebook/bart-large-cnn` выполняет summarization, превращая длинный отчёт в краткое, удобное для оператора резюме.
Summarization применяется только к одному отчёту, что достаточно для демонстрации работы NLP-модели.

In [ ]:
df["date"] = df["timestamp"].dt.date

daily_stats = (
    df.groupby(["area", "date"])
      .agg(
          pm25_mean=("pm25", "mean"),
          pm25_min=("pm25", "min"),
          pm25_max=("pm25", "max"),
      )
      .reset_index()
)

row = daily_stats.iloc[0]

raw_report = (
    f"Daily air quality report for district {row['area']} on {row['date']}.\n"
    f"Average PM2.5 concentration was {row['pm25_mean']:.1f} micrograms per cubic meter. "
    f"The minimum PM2.5 value was {row['pm25_min']:.1f}, and the maximum PM2.5 value was {row['pm25_max']:.1f}. "
    f"This report should help an operator to understand overall air quality conditions in the district during the day."
)

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device_map="auto"
)

# получаем краткое резюме суточного отчёта
summary = summarizer(
    raw_report,
    max_length=60,
    min_length=20,
    do_sample=False
)[0]["summary_text"]

print()
print(summary)

Device set to use cuda:0


Daily air quality report for district Center on 2025-01-01. Average PM2.5 concentration was 25.1 micrograms per cubic meter.
